<a href="https://colab.research.google.com/github/jerkey06/AssetoColab/blob/main/AssettoColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
        █████╗ ███████╗███████╗███████╗████████╗████████╗ ██████╗     ██████╗ ██████╗ ██╗      █████╗ ██████╗
        ██╔══██╗██╔════╝██╔════╝██╔════╝╚══██╔══╝╚══██╔══╝██╔═══██╗    ██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
        ███████║███████╗███████╗█████╗     ██║      ██║   ██║   ██║    ██║     ██║   ██║██║     ███████║██████╔╝
        ██╔══██║╚════██║╚════██║██╔══╝     ██║      ██║   ██║   ██║    ██║     ██║   ██║██║     ██╔══██║██╔══██╗
        ██║  ██║███████║███████║███████╗   ██║      ██║   ╚██████╔╝    ╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
        ╚═╝  ╚═╝╚══════╝╚══════╝╚══════╝   ╚═╝      ╚═╝    ╚═════╝      ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝
```
**Run an Assetto Corsa Server with Mods on Google Colab!**
---
Este notebook te permitirá ejecutar un servidor de Assetto Corsa con soporte para mods y contenido personalizado.

# Configuración de Mods

Primero, necesitamos configurar la estructura de carpetas para los mods y verificar que todo esté en su lugar.

In [4]:
import os
import shutil
import json
from google.colab import files
from google.colab import drive

drive.mount('/content/drive')

base_path = "/content/drive/My Drive/AC-server"
folders = [
    'content/cars',
    'content/tracks',
    'content/weather',
    'system/apps',
    'system/cfg'
]

for folder in folders:
    pull_path = os.path.join(base_path, folder)
    os.makedirs(pull_path, exist_ok=True)
    print(f"Created folder: {pull_path}")

def upload_mod(mod_type):
    """Load mod files to the appropriate directory"""
    print(f'Select the files for {mod_type}...')
    uploaded = files.upload()

    target_dir = os.path.join(base_path, f'content/{mod_type}')
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            import zipfile
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall(target_dir)
            os.remove(filename)
        else:
            shutil.move(filename, os.path.join(target_dir, filename))

    print(f'Successfully uploaded {mod_type} files!')

print('Mods system initialized successfully')

Mounted at /content/drive
Created folder: /content/drive/My Drive/AC-server/content/cars
Created folder: /content/drive/My Drive/AC-server/content/tracks
Created folder: /content/drive/My Drive/AC-server/content/weather
Created folder: /content/drive/My Drive/AC-server/system/apps
Created folder: /content/drive/My Drive/AC-server/system/cfg
Mods system initialized successfully


#Subir Mods

Usa las siguientes celdas para subir tus mods de autos y pistas.

In [ ]:
#upload cars mods
print('Uploading cars mods (files .zip or folders)')
upload_mod('cars')

In [ ]:
#upload traks mods
print('Uploading tracks mods (files .zip or folders)')
upload_mod('tracks')

#Configuracion del servidor (Mods)

In [5]:
import os
import json

def list_available_content():
    """List all available cars and tracks"""
    cars_path = os.path.join(base_path, 'content/cars')
    tracks_path = os.path.join(base_path, 'content/tracks')

    print('Available Cars:')
    cars = [d for d in os.listdir(cars_path) if os.path.isdir(os.path.join(cars_path, d))]
    for car in cars:
        print(f'- {car}')

    print('\nAvailable Tracks:')
    tracks = [d for d in os.listdir(tracks_path) if os.path.isdir(os.path.join(tracks_path, d))]
    for track in tracks:
        print(f'- {track}')

def update_server_config():
    """Update server configuration with selected mods"""
    config_path = os.path.join(base_path, 'server_cfg.json')

    print('\nConfiguring server...')
    server_name = input('Server name: ') or 'Assetto Corsa Server'
    cars = input('Cars (separated by semicolons): ')
    track = input('Track: ')

    config = {
        "SERVER": {
            "NAME": server_name,
            "CARS": cars,
            "TRACK": track,
            "MAX_CLIENTS": 24,
            "ADMIN_PASSWORD": input('Admin password: ') or 'admin',
            "UDP_PORT": 9600,
            "TCP_PORT": 9600,
            "HTTP_PORT": 8081,
            "PICKUP_MODE_ENABLED": 1,
            "LOOP_MODE": 1,
            "SLEEP_TIME": 1,
            "REGISTER_TO_LOBBY": 1,
            "CLIENT_SEND_INTERVAL_HZ": 15,
            "ALLOWED_TYRES_OUT": 4,
            "DAMAGE_MULTIPLIER": 100,
            "FUEL_RATE": 100,
            "TYRE_WEAR_RATE": 100
        }
    }

    with open(config_path, 'w') as f:
        json.dump(config, f, indent=4)

    print('Server configuration updated successfully!')

list_available_content()
update_server_config()

Available Cars:

Available Tracks:

Configuring server...
Server name: prueba
Cars (separated by semicolons): 3
Track: 1
Admin password: 
Server configuration updated successfully!


In [6]:
import os
import json
import requests

# Update packages
!sudo apt update &>/dev/null && echo "apt cache updated" || echo "error updating apt cache"

# Install dependencies
!sudo apt-get install -y lib32gcc1 lib32stdc++6 unzip &>/dev/null && echo "Dependencies installed" || echo "Error installing dependencies"

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/AC-server"

# Verify mods
def verify_mods():
    with open('server_cfg.json') as f:
        config = json.load(f)

    cars = config['SERVER']['CARS'].split(';')
    track = config['SERVER']['TRACK']

    missing = []

    # Verify cars
    for car in cars:
        if not os.path.exists(f'content/cars/{car}'):
            missing.append(f'Car: {car}')

    # Verify track
    if not os.path.exists(f'content/tracks/{track}'):
        missing.append(f'Track: {track}')

    if missing:
        print('WARNING: Missing content:')
        for item in missing:
            print(f'- {item}')
        return False
    return True

# Verify mods before starting
if not verify_mods():
    print('\nPlease make sure all necessary mods are installed before continuing.')
    decision = input('Do you want to continue anyway? (y/n): ')
    if decision.lower() != 'y':
        raise Exception('Server initiation canceled')

# Choose tunnel service
tunnel_service = "argo"  # or "ngrok"
print("Using", tunnel_service, "for connection")

if tunnel_service == "ngrok":
    !pip -q install pyngrok
    from pyngrok import conf, ngrok

    print("Get your token from https://dashboard.ngrok.com/auth")
    import getpass
    authtoken = getpass.getpass()
    !ngrok authtoken $authtoken

    conf.get_default().region = 'us'

    tcp_url = ngrok.connect(9600, 'tcp')
    udp_url = ngrok.connect(9600, 'udp')
    print('Server TCP address:', ((str(tcp_url).split('\"')[1::2])[0]).replace('tcp://', ''))
    print('Server UDP address:', ((str(udp_url).split('\"')[1::2])[0]).replace('udp://', ''))

    print('Starting server...')
    !./acServer

else:
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared-linux-amd64
    print('Starting server...')
    !./cloudflared-linux-amd64 tunnel --url tcp://localhost:9600 & ./acServer


apt cache updated
Error installing dependencies
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/AC-server
- Car: 3
- Track: 1

Please make sure all necessary mods are installed before continuing.
Do you want to continue anyway? (y/n): y
Using argo for connection
--2025-01-07 22:58:16--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.1.0/cloudflared-linux-amd64 [following]
--2025-01-07 22:58:17--  https://github.com/cloudflare/cloudflared/releases/download/2025.1.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: http